# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
import tensorflow as tf
import pandas as pd
import pickle
warnings.filterwarnings('ignore')

In [2]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose

# 1. Load Model

In [3]:
# path to the model
dir_model = "../model/xyz_LSTM_1/"

loaded_model = tf.saved_model.load(dir_model)
inferer = loaded_model.signatures["serving_default"]
inferer??

Signature:      inferer(*args, **kwargs)
Type:           _WrapperFunction
String form:   
ConcreteFunction signature_wrapper_serving_default(*, inputs)
  Args:
    inputs: float32 Tensor, shape=(None, 40, 99)
  Returns:
    {'output_0': <1>}
      <1>: float32 Tensor, shape=(None, 2)
File:           c:\users\ahmad fadli\appdata\roaming\python\python311\site-packages\tensorflow\python\saved_model\load.py
Source:        
class _WrapperFunction(function.ConcreteFunction):
  """A class wraps a concrete function to handle different distributed contexts.

  The reason for wrapping a concrete function is because the _captured_inputs
  fields used for in-replica context and cross-replica context are different.
  When `load()` is called from within a tf.distribute.strategy scope, the
  captured inputs are distributed variables. When using these distributed
  variables during calling the function, we need different approaches when it is
  in-replica and when it is not in-replica. When it is in r

# 2. Setup OpenCV and MediaPipe to Predict

In [4]:
import time

In [14]:
dir_video_test = ('../../data/all videos/video/4.mp4')
cap = cv2.VideoCapture(dir_video_test)
current_stage = ''

i_momentum = 0
i_non_momentum = 0

sequences = []
#initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    # stream the video
    while cap.isOpened():
        ret, frame = cap.read()

        # recolor feed
        image = cv2.resize(frame, (480, 640), interpolation= cv2.INTER_LINEAR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # make detection
        results = pose.process(image)
        
        # recolor image back to BGR for rendering
        image.flags.writeable = False
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # try to predict
        try:
            # model xyz
            row = np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]).flatten()
            sequences.append(row)
            sequences  = sequences[-40:]

            if len(sequences) == 40:
                momentum_class = np.argmax(inferer(inputs=(np.expand_dims(sequences, axis=0)))['output_0'].numpy())
                momentum_prob = np.max(inferer(inputs=(np.expand_dims(sequences, axis=0)))['output_0'].numpy())

            # # model xyz_hand
            # # row = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # # X = pd.DataFrame([row])
            # # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # # model xy
            # # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[]]).flatten()
            # # X = pd.DataFrame([row])
            # # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # # model xy_hand
            # # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # # X = pd.DataFrame([row])
            # # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

                print(momentum_class, momentum_prob)

                if momentum_class == 0 and momentum_prob >= 0.9 :
                    current_stage = 'Nice!'
                    i_non_momentum+=1
                elif momentum_class == 1 and momentum_prob >= 0.9 :
                    current_stage = 'Use momentum!'
                    i_momentum+=1

                # get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
    
                #Display class
                cv2.putText(image, 'Class', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
                #Display Prob
                cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(momentum_prob, 2)), (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        
        except Exception as e:
            print(e)
            
        # Stream vid resultq
        cv2.imshow("Raw Cam Feed", image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1 0.8325726
1 0.810726
1 0.7818894
1 0.7535066
1 0.8064128
1 0.87076193
1 0.9412317
1 0.97903866
1 0.99290943
1 0.99716187
1 0.9992071
1 0.9996812
1 0.9998439
1 0.99994
1 0.99997497
1 0.99998945
1 0.9999941
1 0.9999967
1 0.9999982
1 0.9999991
1 0.9999996
1 0.9999999
1 0.99999994
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 0.9999985
1 0.99938816
0 0.9021613
0 0.95573455
0 0.9794932
0 0.98884773
0 0.9897738
0 0.98776245
0 0.9712855
0 0.9438636
0 0.92392206
0 0.9260676
0 0.892721
0 0.81838614
0 0.7515783
0 0.64443576
1 0.60999376
1 0.63937664
1 0.7508198
1 0.9242799
1 0.98121166
1 0.99448025
1 0.9976641
1 0.99695337
1 0.9983477
1 0.9986583
1 0.99852455
1 0.9973572
1 0.9948496
1 0.9878086
1 0.9391917
1 0.9529402
1 0.9617954
1 0.96764296
1 0.9818659
1 0.99099374
1 0.99443406
1 0.9969226
1 0.99822253
1 0.99901307
1 0.9993955
1 0.9995501
1 0.9997762
1 0.99990207
1 0.9999586
1 0.9999877
1 0.9999955
1 0.99999833

In [49]:
cap.release()
cv2.destroyAllWindows()